### Missionary and Cannibals Code 2

After we have tested in the individual functions, we can put them together. See Missionary and Cannibals 1 first.

Remember to modify the heuristics, and add "self" in the Board methods(). The original code uses List type for the variable Board. Here is converted it to numpy array for convenience, but this requires changing some parts of the Solver3 (add,get) and the Board.solve() function.

In [1]:
from copy import deepcopy
import sys, random
from abc import ABCMeta, abstractmethod
import queue as Q

#
# Basic data structure Stack
#
#
class Stack:
     def __init__(self):
         self.items = []
     def isEmpty(self):
         return self.items == []
     def push(self, item):
         self.items.append(item)
     def pop(self):
         return self.items.pop()
     def peek(self):
         return self.items[len(self.items)-1]
     def size(self):
         return len(self.items)
#
# Basic data structure Queue
# (Use my own Queue instead of import) 
#        
class Queue:
    def __init__(self):
        self.items = []
    def isEmpty(self):
        return self.items == []
    def add(self, item):
        self.items.insert(0,item)
    def get(self):
        return self.items.pop()
    def size(self):
        return len(self.items)


#
# Solver Class: propose moves to the Board Class 
#
#
class Solver(metaclass=ABCMeta):
    """
     ABC=Abstract base class
     Abstract Solver class that searches through the state space, called by the Board class
    """
    @abstractmethod
    def get(self):
         raise NotImplementedError()
    @abstractmethod
    def add(self):
         raise NotImplementedError()
     
class Solver1(Solver):
    """
     Concrete solver: must implement abstract methods from abstract class Solver
     uses a Queue
    """
    def __init__(self):
        self.queue=Queue()
    def get(self):
        return self.queue.get()
    def add(self,board):
        self.queue.add(board)

class Solver2(Solver):
    """
     Concrete solver: must implement abstract methods from abstract class Solver
     Uses a Stack
    """
    def __init__(self):
        self.stack=Stack()
    def get(self):
        return self.stack.pop()
    def add(self,board):
        self.stack.push(board)

class Solver3(Solver):
    """
     Concrete solver 3: must implement abstract methods from abstract class Solver
     Uses the Python built-in Priority Queue
    """

    def __init__(self, goal,n,c):
        self.pq=Q.PriorityQueue()
        self.goal=goal
        self.n=n
        self.c=c
    def get(self):
        if not self.pq.empty():
             (val,board)=self.pq.get()
             return np.array(board)
        else:
             return None
    def add(self, board):
        value=self.heuristic(board)
        b=board.tolist()
        self.pq.put((value,b))
        
    def heuristic(self,board):
        count=0;
        for x in range(self.n*2):
            if self.goal[x]!=board[x]:
                count+=1
        return count

In [2]:
import numpy as np
import itertools

class Board:
    """Model class for the game that stores states about the game board.
       This class also include all the necessary operators.
       Does not handle Animation or screen processses.
       board format: [m,m,m,c,c,c] where m=missionaries, c=cannibals are fixed in position
           m=0 or 1 and c=0 or 1 to represent 0-side or 1-side of the river
       move format: [a,b] where a=index of m or c in board
           a==b then both on the same side
    """    
     
    def __init__(self, n, c):
        self.n=n
        self.c=c
        self.board=np.array([1,1,1,1,1,1]) #MMMCCC -- No BOAT
        self.goal=np.array([0,0,0,0,0,0])
        
        #### CHANGE THE SOLVER METHOD HERE ##
        #self.Solver=Solver1()
        #self.Solver=Solver2()
        self.Solver=Solver3(self.goal,self.n,self.c)
        #####################################
               
    def solve(self): 
        #get board
        #get valid moves
        #put new board state in a data structure
        #repeat
        self.used=[]
        self.Solver.add(self.board)
        board=self.Solver.get()
        #pdb.set_trace()

        found=False
        BOAT=1
        moves=[]
        while not(found):
            self.printBoard(board,moves)
            moves=self.getMoves(board, BOAT)
            print("moves=",moves)
            
            for move in moves:
                print("trying ...",move)
                temp=self.makeMove(board,move, BOAT)
                self.used.append(board)
                if  self.isNotUsed(self.used, temp):
                     self.printBoard(temp,move)
                     self.Solver.add(temp)
                     if np.array_equal(temp,self.goal) :
                          print("Found answer")
                          found=True
                          break
            board=self.Solver.get()
        return #[None,None]
 
    def printBoard(self, board, mv):
        print( "move ",mv, "-> ", board,)
        #print(board)
        print()
        return
    
    def isNotUsed(self,used, board):
        for u in used:
            #print(u,board)
            if np.array_equal(u,board) :
                return False
        return True
    
            
    def makeMove(self,board,move, BOAT):
        test=[b for b in board] # can't do this test=board, have to copy it over
        
        #make move only if not empty move
        if len(move)!=0: #fixed Bug
            if(move[0]>=0):
                test[move[0]]=int(not test[move[0]])
            if(move[1]>=0):
                test[move[1]]=int(not test[move[1]])
            #print(test)    

            # Only update BOAT if a move has been made.
            if(not np.array_equal(test,board)):
                BOAT=int(not(BOAT))
                
        return np.array(test)
            
    def isValidMove(self, board, move):
        '''
        move format: [a b]
        a and b is the index of the passenger in board
        board[a]==board[b] if both are on the same river side
        board[a]!=board[b] is not valid since they are not on the same side
        '''
        val1=False

        test=[b for b in board] # can't do this test=board, have to copy them
        #print(move[0],move[1],"<<--")
        
        #move must from same side
        if(board[move[0]]==board[move[1]]): #fixed bug
            if(move[0]>=0): #valid index not -1
                test[move[0]]=int(not test[move[0]]) #update board copy
            if(move[1]>=0): #valid index not -1
                test[move[1]]=int(not test[move[1]]) #valid index not -1
        
        #we can't just count zero or one because they are coded by position
        m, mcounts = np.unique(test[:3], return_counts=True) #count how many 0 and 1
        c, ccounts = np.unique(test[3:], return_counts=True) #count how many 0 and 1

        mm=dict(zip(m, mcounts)) # make it into dictionary pairs
        cc=dict(zip(c, ccounts)) # make it into dictionary pairs

        if(mm.get(1) and cc.get(1)):
            if(mm.get(1)>=cc.get(1)): #valid if number of M >= C on Bank=1
                val1=True
            else:
                return False
        if(mm.get(0) and cc.get(0)):
            if(mm.get(0)>=cc.get(0)): #valid if number of M >= C on Bank=0
                val1=True
            else:
                return False
        if(mm.get(0) and cc.get(1)):
            if(mm.get(0)==3):
                return True
        if(mm.get(1) and cc.get(0)):
            if(mm.get(1)==3):
                return True
        return val1 

    def getMoves(self,board, BOAT):
        a,b=-1,-1
        moves=[]
        
        zero=np.where(board==0)[0] ### board has to be np.aray
        one=np.where(board==1)[0]
        
        if(BOAT==0):
            if(len(zero)==1):  
                a=zero[0] #only 1 choice
                moves=[[a,b]]
            elif(len(zero)==2):  
                a=zero[0] #only 1 choice
                b=zero[1] #only 1 choice
                moves=[[a,b]]
            else:#select all combinations not randomly
                for i in itertools.combinations(zero,2):
                    moves.append([i[0],i[1]])

        elif(BOAT==1):
            if(len(one)==1):  
                a=one[0] #only 1 choice
                moves=[[a,b]]
            elif(len(one)==2):  
                a=one[0] #only 1 choice
                b=one[1] #only 1 choice
                moves=[[a,b]]
            else:
                for i in itertools.combinations(one,2):
                    moves.append([i[0],i[1]])

        validmoves=[m for m in moves if self.isValidMove(board,m)==True]
        #validmoves could be empty
        return validmoves


In [3]:
puzzle=Board(3,2)
puzzle.solve()

move  [] ->  [1 1 1 1 1 1]

moves= [[0, 3], [0, 4], [0, 5], [1, 3], [1, 4], [1, 5], [2, 3], [2, 4], [2, 5], [3, 4], [3, 5], [4, 5]]
trying ... [0, 3]
move  [0, 3] ->  [0 1 1 0 1 1]

trying ... [0, 4]
move  [0, 4] ->  [0 1 1 1 0 1]

trying ... [0, 5]
move  [0, 5] ->  [0 1 1 1 1 0]

trying ... [1, 3]
move  [1, 3] ->  [1 0 1 0 1 1]

trying ... [1, 4]
move  [1, 4] ->  [1 0 1 1 0 1]

trying ... [1, 5]
move  [1, 5] ->  [1 0 1 1 1 0]

trying ... [2, 3]
move  [2, 3] ->  [1 1 0 0 1 1]

trying ... [2, 4]
move  [2, 4] ->  [1 1 0 1 0 1]

trying ... [2, 5]
move  [2, 5] ->  [1 1 0 1 1 0]

trying ... [3, 4]
move  [3, 4] ->  [1 1 1 0 0 1]

trying ... [3, 5]
move  [3, 5] ->  [1 1 1 0 1 0]

trying ... [4, 5]
move  [4, 5] ->  [1 1 1 1 0 0]

move  [[0, 3], [0, 4], [0, 5], [1, 3], [1, 4], [1, 5], [2, 3], [2, 4], [2, 5], [3, 4], [3, 5], [4, 5]] ->  [0 1 1 0 1 1]

moves= [[1, 2], [1, 4], [1, 5], [2, 4], [2, 5]]
trying ... [1, 2]
move  [1, 2] ->  [0 0 0 0 1 1]

trying ... [1, 4]
move  [1, 4] ->  [0 0 1 0 0 1]

### Answer

Based on the code results:

(0,3)->(1,2)->(4,5)

    111111 move->(0,3)
    011011 move->(1,2)
    000011 move->(4,5)
    000000 finish
    
(Please double check if correct)